In [ ]:
# Install the required libraries
!pip install prophet

In [ ]:
# Dependencies
# NOTE: We might not use all of these. I just improrted everything I can think of for now. We'll delete the ones we don't need later
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import json
import path

In [ ]:
#Read data into the notebook
linkedin_postings_df = pd.read_csv('./data_sets/postings.csv').dropna()
machine_learning_jobs_df = pd.read_json('./data_sets/job_data.json', lines=True)

# Normalize, clean, massage, and combine data for ease of processing

# Cast all job skills to lower case strings to standardize string matching later
linkedin_postings_df['job_skills'] = linkedin_postings_df['job_skills'].apply(lambda item: item.lower().split(', '))

***Introduction*** 
The goal of this exploratory data analysis is to characterize and investigate the growth of machine learning as a job skill. We are interested in looking at this topic along a number angles. TBC....

QUESTION Geography 

In [ ]:
# Insert Question 1 analysis and visualizations here. Insert new cells if necessary 

Q1 Summary \[INSERT SUMMARY HERE] ... write a little about what the findings above seem to indicate about question 1

Question 2 Relative proportions ML to not-ML

In [ ]:
# Insert question 2 analysis etc here. Insert new cells if necessary

Q2 Summary \[INSERT SUMMARY HERE]

Question Job Skills

Question Seniority/Job level

In [ ]:
# etc 

Question 5 Industry demand 

In [ ]:
QUESTION: AI MODELS

In [ ]:
# Read data into notebook
ai_index_df = pd.read_csv('./data_sets/ai_job_index.csv')

# Visualzie and explore the dataframe
ai_index_df

In [ ]:
# Analyze the "Domain" column for unique values.
ai_index_df['Domain'].unique()

In [ ]:
#Rename "Domain" column to "Sectors" to match notebook analysis
ai_index_df = ai_index_df.rename(columns={'Domain':'Sector'})
ai_index_df

In [ ]:
# Clean the data by removing leading/trailing spaces by removing special characters
ai_index_df['Sector'] = ai_index_df['Sector'].replace('[^\w\s]', '', regex=True)
ai_index_df

In [ ]:
# Define keywords to group Sectors in preparation for further analysis and aggregation.
sector_grouping = {'Communication  PR':'Media', 'Data  IT':'Tech', 'Administrative  Clerical':'Clerical',
                   'Leadership  Strategy':'Consulting', 'Medical  Healthcare':'Healthcare',
                   'Supply Chain  Logistics':'Manufacturing', 'Law Enforcement':'Law Enforcement', 
                   'Construction':'Construction','Sales  Marketing':'Marketing', 'Hospitality':'Hospitality'}

ai_index_df['Sector'] = ai_index_df['Sector'].replace(sector_grouping)
ai_index_df

In [ ]:
# Rename columns for further analysis and clarity
ai_index_df = ai_index_df.rename(columns={'Tasks':'Human Tasks'})
ai_index_df

In [ ]:
# Drop colummns that don't pertain to further aggregation and visualization
ai_index_df = ai_index_df.drop(columns=['Job titiles','AI Impact','AI_Workload_Ratio'])
ai_index_df

In [ ]:
# Groupby "Sector" and use the sum() aggregation function on the "AI Models" column to calculate the total number of AI models per Sector
# Visualize data for further analysis.
models_by_sectors = ai_index_df.groupby('Sector')['AI models'].sum()
models_by_sectors_df = models_by_sectors.reset_index()
models_by_sectors_df = models_by_sectors_df.rename(columns={'AI models':'AI Models by Sector'})
plt.bar(models_by_sectors_df['Sector'], models_by_sectors_df['AI Models by Sector'], color='orange')
plt.xticks(rotation=90)
plt.ylabel('Total AI Models')
plt.tight_layout()
plt.savefig('AI_Models', dpi=300)
plt.show()

This bar graph provides a visual representation of the prevalence of AI Models in the workforce, with at least 800,000 AI models being used in every "Sector" of the dataset.

In [ ]:
# Groupby "Sector" and use the sum() aggregation function on the "Human Tasks" column to calculate the total amount of Human Tasks per Sector
# Visualize data for further analysis.
tasks_by_sectors = ai_index_df.groupby('Sector')['Human Tasks'].sum()
tasks_by_sectors_df = tasks_by_sectors.reset_index()
plt.bar(tasks_by_sectors_df['Sector'], tasks_by_sectors_df['Human Tasks'], color='red')
plt.xticks(rotation=90)
plt.ylabel('Total Human Tasks')
plt.tight_layout()
plt.savefig('Human_Tasks.png', dpi=300)
plt.show()

Comparatively, this bar graph provides a visual representation of the "Total Human Tasks" by "Sector" replaceable by AI Models. Calculating the proportions of "Human Tasks" to "AI Models" will provide more insight.

In [ ]:
# Merge the AI Models and Human Tasks dataframe and calculate the proportion of Human Tasks to AI Models
ai_workload = pd.merge(tasks_by_sectors_df, models_by_sectors_df, on='Sector', how='outer')
ai_workload['AI Proportion'] = ai_workload['Human Tasks']/ai_workload['AI Models by Sector']
ai_workload

In [ ]:
# Use pandas statistical analysis to determine the mean AI workload based on the "Proportion" column.
ai_workload['AI Proportion'].describe()

In [ ]:
# Use statistical visualization to confirm findings
plt.boxplot(ai_workload['AI Proportion'])
plt.ylabel('AI Workload')
plt.tight_layout()
plt.savefig('AI_Workload.png', dpi=300)
plt.show()

Based on the total proportions of "Human Tasks" to "AI Models", the statistical analysis revealed an average AI model workload of 22%, meaning for every individual human task, there are 4-5 AI models being trained as replacements. This shift to a more technological landscape raises many questions while providing insight on future employment trends.